In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()


False

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"C:\Users\ICA-4047\Desktop\llmops\merged dataset.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'C:\\Users\\ICA-4047\\Desktop\\llmops\\merged dataset.pdf', 'page': 0}, page_content=' \nAccount Opening Checklist\nCentralized Account Opening\nOperations Group\n'),
 Document(metadata={'source': 'C:\\Users\\ICA-4047\\Desktop\\llmops\\merged dataset.pdf', 'page': 1}, page_content="                                                           \n \nAccount Opening Checklist  \n \n                                                                                                                                                                             Page 4  \n \n \n \n \n \n \nComprehensive  Check -list for Opening of Accounts  \nS. No  General Documentation  of Natural Person  required for Account Opening  \n1.  A photocopy of any one of the following valid identity documents duly attested by Gazetted \nOfficer/ Chairman/ administrator or an Officer of the Branch after original seen.  \n \na) CNIC/NICOP/SNIC /SNICOP /SNICOP/ POC/ARC /POR/Juvenile Card  issued

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': 'C:\\Users\\ICA-4047\\Desktop\\llmops\\merged dataset.pdf', 'page': 0}, page_content='Account Opening Checklist\nCentralized Account Opening\nOperations Group'),
 Document(metadata={'source': 'C:\\Users\\ICA-4047\\Desktop\\llmops\\merged dataset.pdf', 'page': 1}, page_content='Account Opening Checklist  \n \n                                                                                                                                                                             Page 4  \n \n \n \n \n \n \nComprehensive  Check -list for Opening of Accounts  \nS. No  General Documentation  of Natural Person  required for Account Opening  \n1.  A photocopy of any one of the following valid identity documents duly attested by Gazetted \nOfficer/ Chairman/ administrator or an Officer of the Branch after original seen.  \n \na) CNIC/NICOP/SNIC /SNICOP /SNICOP/ POC/ARC /POR/Juvenile Card  issued by NADRA/  \nCRC/Form B  etc. \nb) Passport, having valid visa on it

In [9]:
## Convert Data Into Vectors and store in DB
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_community.embeddings import OpenAIEmbeddings
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(docs,OpenAIEmbeddings())

In [10]:

db

In [11]:
query="eligibility criteria "
result=db.similarity_search(query)
result[0].page_content

'Eligibility Criteria\nEligibility Criteria\nAll Pakistani nationals/Foreign nationals with company letter of comfort (Approved companies)\nSalaried Individuals, Self Employed Businessmen/Professionals, others (Landlords/ Housewives & Retired\nIndividuals) with consistent rental income/foreign remittances\nAge │ min: 21 years & max: 60 years for Salaried │ min: 21 years & max: 65 years for self-employed\nprofessionals and businessmen\nIncome | Salaried : Min. gross monthly salary of PKR. 50,000/-\nSelf Employed Professionals: Min. monthly average balance of PKR. 75,000/-\nSelf Employed Businessman: Min. monthly average balance of PKR. 75,000/-\n\uf075\n(https://www.bankalfalah.com)\uf1429/3/24, 1:04 PM Eligibility Criteria – Bank Alfalah\nhttps://www.bankalfalah.com/info/loans/eligibility-criteria-7/ 1/1'

In [12]:
groq_api_key=os.environ['GROQ_API_KEY']
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="mixtral-8x7b-32768"
)


from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [14]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCEF459EB0>, search_kwargs={})

In [15]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [16]:
response=retrieval_chain.invoke({"input":"CREDIT CARDS"})

In [17]:
response['answer']

'Based on the provided context, the text discusses various types of credit cards and related services offered by Bank Alfalah in Pakistan. Here are some key points about the credit cards:\n\n1. Bank Alfalah offers various types of credit cards, including Visa, MasterCard, and American Express cards, with different features and annual fees.\n2. The credit cards offer different interest rates on cash advances and retail transactions. For example, the Visa Classic Credit Card has an interest rate of 3.83% per month (46% Annual Percentage Rate) on cash advances and retail transactions.\n3. The bank offers a Credit on Phone (COP) facility, which enables the customer to purchase goods and services and transfer the facility/transaction amounts to a Step-by-Step payment plan (SBS) and repay the amount in equal monthly installments.\n4. The bank offers up to six supplementary cards to the credit cardholders, who can share the benefits and privileges of their Bank Alfalah Credit Card with their 